In [ ]:
from clearml import Dataset

In [ ]:
dataset = Dataset.create(
    dataset_name="Twitter Sentiment Analysis dataset",
    dataset_project="Twitter Sentiment Analysis",
    dataset_version="1.0",
    description="Data from kaggle project",
)
dataset.finalize()

In [ ]:
import polars as pl

for index, batch in enumerate(
    pl.read_csv(
        "data/twitter_training.csv",
        has_header=False,
        new_columns=["ID", "Entity", "Sentiment", "Tweet content"],
    )
    .with_row_index()
    .with_columns(pl.col("index") // 25000)
    .partition_by("index")
):
    batch.write_csv(f"data/raw/t_batch_{index}.csv")
    polaritu_distrib = batch.group_by("Sentiment").len()
    dataset = Dataset.create(
        dataset_name="Twitter Sentiment Analysis dataset",
        dataset_project="Twitter Sentiment Analysis",
        parent_datasets=[dataset],
        dataset_version=f"1.{index}",
        description="Data from kaggle project",
    )
    dataset.add_files(path=f"data/raw/t_batch_{index}.csv")
    dataset.get_logger().report_table(
        "Dataset Preview", "Dataset Preview", table_plot=batch.head(5).to_pandas()
    )
    dataset.get_logger().report_histogram(
        title="Sentiment distribution",
        series="Sentiment distribution",
        values=polaritu_distrib["len"].to_list(),
        xlabels=polaritu_distrib["Sentiment"].to_list(),
        yaxis="Number of samples",
    )
    dataset.upload()
    dataset.finalize()